# Создал этот ноутбук для проверки простеньких сеток на синтетических датасетах / кусках датасетов

- Векторы V все почти что коллинеарны, когда их много - стоит уже посмотреть на $p_i$ и $r_{cut_{i}}$

- 

In [48]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import torch.optim as optim
from torch.optim import lr_scheduler

import seaborn as sns
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

In [49]:
def function(X):
    return 5 * X

In [50]:
path = '4_dataset_K_3.pt'

In [51]:
class CFG:
    '''

    All hyperparameters are here

    '''

    N = int(path.split('_')[0])     # число атомов
    K = int(path.split('_')[-1].split('.')[0])     # можно называть это разрешением...чем число больше, тем больше размеры матрицы для атомов, фактически это число элементов в наборах p и r_cut

    L = 2 * N ** (1 / 3) # размер одной клетки при моделировании

    r_cut = np.random.uniform(low=5, high=10, size=K).copy()
    p = np.random.uniform(low=1, high=3, size=K).copy()
    N_neig= N - 1 if N != 2 else 1

    # train_bs = 8
    # val_bs = 16
    batch_size = 64

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    f_threshold = 5    # Если сила по какой-то координате превышает это значение, то строчка исключается, совсем маленьких по модулю сил быть не должно, если что при генерации просто r_cut поменьше надо делать
    coord_threshold = L     # Если вдруг очень большие расстояния, то надо выкидывать
    f_min_threshold = 0.05
    #
    output_size = K     # Размерность аутпута модели

In [52]:
def create_dataloaders(train_dataset, val_dataset, train_bs=64, val_bs=64, fold=None):
    '''

    Returns train_loader, val_loader

    fold: will be used in cross validation, when I will implement it

    '''
    
    train_loader = DataLoader(dataset=train_dataset, batch_size=train_bs, shuffle=True)

    val_loader = DataLoader(dataset=val_dataset, batch_size=val_bs, shuffle=False)

    return train_loader, val_loader

In [53]:
def plot_k_dim_f_norm_from_x_norm(data_sample, figsize=(15, 10), fontsize=18):
    '''

    Функция строит зависимость таргета от X

    data_sample - итерируемый объект из тьюплов вида: (x, f_k_dim, f_3d, pinv_A)

    '''
    plt.figure(figsize=figsize)
    x = torch.stack([elem[0][0].squeeze() for elem in data_sample])

    y = torch.stack([elem[1][0].squeeze() for elem in data_sample])

    plt.scatter(x, y, label='y(x)')
    plt.title('Зависимость $Y_{target}(X)$', fontsize=fontsize)

    None

In [54]:
# class MultiOutputMSELoss(nn.MSELoss):
#     '''

#     Custom loss that calculates average over batch loss for multidim MSE - sum of MSE for components

#     Example:
#     |
#     |    Loss = MultiOutputMSELoss()
#     |
#     |    a = torch.ones((8, 3))      # it is batch of 8 vectors of size 3
#     |    b = torch.zeros((8, 3))
#     |
#     |    Loss(a, b, batch_size=8) -> 3

#     '''

#     def forward(self, input, target, batch_size=CFG.batch_size):
#         '''
#         оно при reduction='mean' делит на произведение всех размерностей
#         '''
#         # при очень большом размере батча последние батчи будут например размера 128 вместо 256, поэтому просто умножать на батч сайз неправильно, могут быть другого размера

#         return F.mse_loss(input, target, reduction='sum') / input.size(0)   # или эквивалентно делать reduction='mean' и умножать на input.size()[-1] - length of output

In [55]:
def make_predictions_and_plot(model, X_matrices=None, Y_target=None, figsize=(30, 20), fontsize=20, criterion=nn.MSELoss(), data=None, same_axis=False):
    '''

    Строит предсказанную и тагрет зависимости

    Можно подавать либо отлельно X_matrices, Y_target либо подать датасет из тьюплов: (X, f_k_dim, f_3d, A_pinv)

    Будет подаваться 3 итерируемых объекта: набор(батч или кусок датасета) из "матриц", предсказания, таргеты

    Качество

    '''

    if data:
        X_matrices, Y_target, _, _ = list(map(lambda x: torch.stack(x), list(zip(*data))))
        
    Y_pred = model(X_matrices)
    metric = criterion(Y_pred, Y_target)
    names = ['predicted', 'target']
    Ys = [Y_pred, Y_target]
    metric = criterion(Y_pred, Y_target)
    
    if CFG.K != 1:
        print(
            f'Metric: {metric}'
        )
        return

    plt.figure(figsize=figsize)
    fig = plt.gcf()
    fig.suptitle(f'Metric: {metric}', fontsize=22)

    for (i, name) in enumerate(names):
        if not same_axis:
            plt.subplot(1, 2, i + 1)

        plt.scatter(X_matrices.squeeze().detach().numpy(), Ys[i].squeeze().detach().numpy(), label='Y(X)')

        plt.xlabel('X', fontsize=18)
        plt.ylabel('Y', fontsize=18)
        plt.legend(loc='best', fontsize=fontsize)
        plt.title(f'{name}', fontsize=18)
        plt.grid(alpha=0.4)

    plt.show()

In [56]:
def fit_epoch(model, train_loader, criterion, optimizer, scheduler, scaler, show_3D_quality):
    '''

    Функция обучения по всем батчам 1 раз (1 эпоха)

    scaler: gradient scaler from torch.amp, попозже добавлю обучение с ним

    В данной версии: (X, f_k_dim)

    Лосс выводится для k-мерного предсказания, а mse считается по 3D вариантам, однако при K=1 3d и 1d MSE совпадают

    '''
    model.train()

    running_loss = 0.0
    running_MSE = 0
    processed_size = 0

    for inputs, labels, pinv_As, labels_3D in train_loader:
        optimizer.zero_grad()

        inputs = inputs.to(CFG.device)
        labels = labels.to(CFG.device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        if scheduler:
            scheduler.step()

        if show_3D_quality:
            # print(pinv_As.size(), torch.unsqueeze(outputs.to(torch.float), 2).size())
            outputs_3D = torch.bmm(pinv_As, torch.unsqueeze(outputs.to(torch.float), 2)).to(CFG.device)    # используются для вычисления MSE метрики уже на 3D векторах силы
            outputs_3D = torch.squeeze(outputs_3D, -1)
            running_MSE += F.mse_loss(input=outputs_3D, target=labels_3D, reduction='sum').item()

        running_loss += loss.item() * inputs.size(0)    # при очень большом размере батча последние два батча будут например размера 128 вместо 256, поэтому просто умножать на батч сайз неправильно, могут быть другого размера
        processed_size += inputs.size(0)

    # print(labels)
    train_loss = running_loss / processed_size
    if show_3D_quality:
        train_MSE = running_MSE / processed_size
        return train_loss, train_MSE
    
    return train_loss

In [57]:
def eval_epoch(model, val_loader, criterion, show_3D_quality):
    '''

    Одна эпоха по val выборке

    '''

    model.eval()
    
    running_loss = 0.0
    running_MSE = 0
    processed_size = 0

    for inputs, labels, pinv_As, labels_3D in val_loader:

        inputs = inputs.to(CFG.device)
        labels = labels.to(CFG.device)

        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        if show_3D_quality:
            outputs_3D = torch.bmm(pinv_As, torch.unsqueeze(outputs.to(torch.float), 2)).to(CFG.device)    # используются для вычисления MSE метрики уже на 3D векторах силы
            outputs_3D = torch.squeeze(outputs_3D, -1)
            running_MSE += F.mse_loss(input=outputs_3D, target=labels_3D, reduction='sum').item()

        running_loss += loss.item() * inputs.size(0)    # при очень большом размере батча последние два батча будут например размера 128 вместо 256, поэтому просто умножать на батч сайз неправильно, могут быть другого размера
        processed_size += inputs.size(0)

    # print(f' outputs:\n{outputs}, \n labels: \n {labels}')
    
    val_loss = running_loss / processed_size
    if show_3D_quality:
        val_MSE = running_MSE / processed_size
        return val_loss, val_MSE

    return val_loss

In [58]:
def train(train_loader, val_loader, model, optimizer, scheduler=None, epochs=10, scaler=None, criterion=nn.MSELoss(), show_3D_quality=False):
    '''

    Basic option: calculation loss on K-dimensional outputs, but MSE metric on 3D outputs, after the matrix is applied

    loss_on_k_projections: calculate loss'
    
    '''

    history = []
    if show_3D_quality:
        log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} val_loss: {v_loss:0.4f} train_mse: {t_mse:0.4f} val_mse: {v_mse:0.4f}"
    else:
        log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} val_loss: {v_loss:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:

        for epoch in range(epochs):
            if show_3D_quality:
                train_loss, train_mse = fit_epoch(model, train_loader, criterion, optimizer, scheduler, scaler, show_3D_quality)
                val_loss, val_mse = eval_epoch(model, val_loader, criterion, show_3D_quality)
            else:
                train_loss = fit_epoch(model, train_loader, criterion, optimizer, scheduler, scaler, show_3D_quality)
                val_loss = eval_epoch(model, val_loader, criterion, show_3D_quality)
            history.append((train_loss, val_loss))
            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch + 1, t_loss=train_loss, v_loss=val_loss))
            
    return history

In [59]:
class SingleNet(nn.Module):
    '''

    Класс одиночной нейронной сети

    '''
    def __init__(self, output_size, activation=nn.ReLU(), flattened_size=CFG.K * CFG.K):
        '''
        
        FC_type: тип полносвязных слоев: 'regular' / 'simple

        convolution: сверточная часть сети

        '''
        super().__init__()

        self.FC = nn.Sequential(
            nn.Linear(flattened_size, 64),
            activation,
            # nn.Dropout(0.3),
            nn.BatchNorm1d(64),

            nn.Linear(64, 128),
            activation,
            # nn.Dropout(0.3),
            nn.BatchNorm1d(128),
            
            nn.Linear(128, 256),
            activation,
            # nn.Dropout(0.3),
            nn.BatchNorm1d(256),
            nn.Linear(256, output_size),
        )

        # self.FC = nn.Linear(flattened_size, output_size)

    def forward(self, x):
        # x - is batch of matrices KxK

        # Здесь происходят какие-то там свертки, пуллинги и тп..

        x = self.FC(x)

        return x

In [60]:
def recieve_loaders(take_one_projection_for_data=None, path=None, cut_size=None):
    '''
    returns: (dataet, train_loader, val_loader), if u pass path -> returns loader from tensor dataset from

    может создать датасет у которого всего одна проекция взята за таргет из уже существующего датасета или по пути на файл:
    take_one_projection_for_data - указываем в этом параметре номер координатной проекции (нумерация от 1)
    '''
    if path:
        N = int(path.split('_')[0])
        K = int(path.split('_')[-1].split('.')[0])

        dataset = torch.load(str(N) + '_dataset_K_' + str(K) + '.pt')
        dataset = [(elem[0], elem[1], elem[2], elem[3]) for elem in dataset]

        if take_one_projection_for_data:
 
            dataset = [(elem[0], elem[1][take_one_projection_for_data - 1].unsqueeze(dim=0), elem[2], elem[3]) for elem in dataset]

        train_data, val_data = train_test_split(dataset, test_size=0.33, random_state=42)
        if cut_size:
            train_data = train_data[:cut_size]
            val_data = val_data[:cut_size]
        train_dataloader, val_dataloader = create_dataloaders(train_data, val_data)
        return train_data, val_data, train_dataloader, val_dataloader
    
    X = []
    Y = []

    for _ in range(500):
        # X = (torch.rand(1)).squeeze().unsqueeze(dim=0)
        x = (torch.rand(K))
        y = function(x)
        X.append(x)
        Y.append(y)

    X_train, X_val, Y_train, Y_val = train_test_split(X, Y, random_state=42, train_size=0.8)

    X_train = torch.stack(X_train)
    X_val = torch.stack(X_val)
    Y_train = torch.stack(Y_train)
    Y_val = torch.stack(Y_val)

    train_data = TensorDataset(X_train, Y_train)
    val_data = TensorDataset(X_val, Y_val)

    train_dataloader = DataLoader(train_data, batch_size=128)
    val_dataloader = DataLoader(val_data, batch_size=128)

    return train_data, val_data, train_dataloader, val_dataloader

In [61]:
train_data, val_data, train_dataloader, val_dataloader = recieve_loaders(
    take_one_projection_for_data=None,
    path=path,
    cut_size=None
    )

In [62]:
train_data

[(tensor([0.0267, 0.0000, 0.0000, 0.0000, 2.2240, 0.0000, 0.0000, 0.0000, 0.6544]),
  tensor([-0.2532,  0.1094,  0.0175]),
  tensor([[ 1.,  0.,  0.],
          [ 0., -1.,  0.],
          [ 0.,  0., -1.]]),
  tensor([-0.2532, -0.1094, -0.0175])),
 (tensor([0.9146, 0.0000, 0.0000, 0.0000, 1.3844, 0.0000, 0.0000, 0.0000, 1.6221]),
  tensor([-0.0767,  0.1315,  0.0096]),
  tensor([[ 1.,  0.,  0.],
          [ 0.,  1.,  0.],
          [ 0.,  0., -1.]]),
  tensor([-0.0767,  0.1315, -0.0096])),
 (tensor([2.2047, 0.0000, 0.0000, 0.0000, 0.6540, 0.0000, 0.0000, 0.0000, 0.3429]),
  tensor([-0.1232, -0.2672,  0.0225]),
  tensor([[-1.,  0.,  0.],
          [ 0.,  1.,  0.],
          [ 0.,  0.,  1.]]),
  tensor([ 0.1232, -0.2672,  0.0225])),
 (tensor([1.1433, 0.0000, 0.0000, 0.0000, 0.2441, 0.0000, 0.0000, 0.0000, 2.0034]),
  tensor([ 0.3067, -0.2076, -0.1531]),
  tensor([[1., 0., 0.],
          [0., 1., 0.],
          [0., 0., 1.]]),
  tensor([ 0.3067, -0.2076, -0.1531])),
 (tensor([0.0113, 0.0000,

Сама частица, отступаем от нее вектор силы

In [63]:
# N=2, K=1:

# flattened_size = CFG.K ** 2
# output_size = CFG.K

# model = nn.Sequential(
#     nn.Linear(flattened_size, 8),
#     nn.ReLU(),
#     # nn.Dropout(0.3),
#     nn.BatchNorm1d(8),

#     nn.Linear(8, 32),
#     nn.ReLU(),
#     # nn.Dropout(0.3),
#     nn.BatchNorm1d(32),

#     nn.Linear(32, 128),
#     nn.ReLU(),
#     # nn.Dropout(0.3),
#     nn.BatchNorm1d(128),
#     nn.Linear(128, output_size),
#     ).to(CFG.device)

# optimizer = optim.Adam(model.parameters(), lr=5e-3, betas=(0.9, 0.999), weight_decay=0.01)

# exp_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [64]:
model = SingleNet(
    output_size=CFG.K,

    # activation=nn.Sigmoid(),
    activation=nn.ReLU(),

).to(CFG.device)

# model = nn.Linear(1, 1)

# model = nn.Sequential(
#     nn.Linear(CFG.K ** 2, 8),
#     nn.ReLU(),
#     # nn.Dropout(0.5),
#     # nn.BatchNorm1d(8),

#     nn.Linear(8, 32),
#     nn.ReLU(),
#     # nn.Dropout(0.5),
#     # nn.BatchNorm1d(32),

#     nn.Linear(32, CFG.K)
# )

optimizer = optim.Adam(model.parameters(), lr=5e-4, betas=(0.9, 0.999), weight_decay=0.01)

# scheduler.step нужно первый раз делать обязательно после optimizer.step, потому что иначе мы просто пропустим первый шаг scheduler
exp_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.95)

model

SingleNet(
  (FC): Sequential(
    (0): Linear(in_features=9, out_features=64, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=64, out_features=128, bias=True)
    (4): ReLU()
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=128, out_features=256, bias=True)
    (7): ReLU()
    (8): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=256, out_features=3, bias=True)
  )
)

In [65]:
history = train(
    train_loader=train_dataloader, val_loader=val_dataloader, model=model, optimizer=optimizer,
    
    scheduler=exp_scheduler,
    scaler=None,
    
    # criterion=MultiOutputMSELoss(),     # on K=1 it's the same as nn.MSELoss
    criterion=nn.MSELoss(),
    # criterion = GaussianNLLLossWithReadyVar(),

    # show_3D_quality=True,    # ПОКА ЧИСТО ТЕСТЮ НА ОДНОЙ ПРОЕКЦИИ И ТАМ РАЗМЕРНОСТИ НЕ СОВПАДАЮТ
    
    epochs=10
)

epoch:  10%|█         | 1/10 [00:00<00:03,  2.65it/s]


Epoch 001 train_loss: 0.0548 val_loss: 0.0474


epoch:  20%|██        | 2/10 [00:00<00:03,  2.21it/s]


Epoch 002 train_loss: 0.0450 val_loss: 0.0446


epoch:  30%|███       | 3/10 [00:01<00:02,  2.51it/s]

...


Epoch 008 train_loss: 0.0425 val_loss: 0.0430


epoch:  90%|█████████ | 9/10 [00:03<00:00,  2.76it/s]


Epoch 009 train_loss: 0.0426 val_loss: 0.0430


epoch: 100%|██████████| 10/10 [00:03<00:00,  2.69it/s]


Epoch 010 train_loss: 0.0424 val_loss: 0.0429


In [66]:
def plot_history(history, figsize=(40, 15)):
    '''

    history: [(train_loss, train_MSE, val_loss, val_MSE), ...]

    '''

    names = ['Losses', 'Metrics']
    
    figure = plt.figure(figsize=figsize)

    train_loss, train_metric, val_loss, val_metric = list(zip(*history))

    parameters = {'Losses': [train_loss, val_loss], 'Metrics': [train_metric, val_metric]}

    for i, name in enumerate(names):

        plt.subplot(1, 2, i + 1)
        plt.plot(range(1, len(history) + 1), parameters[name][0], label='train_loss')
        plt.plot(range(1, len(history) + 1), parameters[name][1], label='val_loss')
        
        plt.title(names[i], fontsize=18)
        plt.xlabel('epoch', fontsize=16)
        plt.ylabel('value', fontsize=16)     
        plt.xticks(np.arange(1, len(history) + 1, 1))
        # plt.yticks(np.arange(1, len(history) + 1, 1))
        plt.legend(loc='best', fontsize=16)
        plt.grid(alpha=0.4)
    
    plt.show()

In [67]:
make_predictions_and_plot(model=model, data=val_data, same_axis=True)

Metric: 0.04290896654129028


In [68]:
plot_history(history=history)

ValueError: not enough values to unpack (expected 4, got 2)

<Figure size 2880x1080 with 0 Axes>

In [69]:
train_data[0]

(tensor([0.0267, 0.0000, 0.0000, 0.0000, 2.2240, 0.0000, 0.0000, 0.0000, 0.6544]),
 tensor([-0.2532,  0.1094,  0.0175]),
 tensor([[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]]),
 tensor([-0.2532, -0.1094, -0.0175]))

In [ ]:
x_t = next(iter(val_dataloader))[0]
y_t = next(iter(val_dataloader))[1]
x_t, y_t

(tensor([[1.0953, 0.0000, 0.0000, 0.0000, 1.1905, 0.0000, 0.0000, 0.0000, 1.6597],
         [1.2229, 0.0000, 0.0000, 0.0000, 0.2046, 0.0000, 0.0000, 0.0000, 1.9583],
         [1.1480, 0.0000, 0.0000, 0.0000, 0.2374, 0.0000, 0.0000, 0.0000, 1.9986],
         [1.0816, 0.0000, 0.0000, 0.0000, 1.2283, 0.0000, 0.0000, 0.0000, 1.6457],
         [2.1806, 0.0000, 0.0000, 0.0000, 0.7610, 0.0000, 0.0000, 0.0000, 0.2261],
         [1.2087, 0.0000, 0.0000, 0.0000, 0.2091, 0.0000, 0.0000, 0.0000, 1.9663],
         [0.8466, 0.0000, 0.0000, 0.0000, 1.4431, 0.0000, 0.0000, 0.0000, 1.6054],
         [1.2297, 0.0000, 0.0000, 0.0000, 0.2057, 0.0000, 0.0000, 0.0000, 1.9560],
         [1.1668, 0.0000, 0.0000, 0.0000, 0.2300, 0.0000, 0.0000, 0.0000, 1.9901],
         [0.0602, 0.0000, 0.0000, 0.0000, 2.2518, 0.0000, 0.0000, 0.0000, 0.5666],
         [2.1665, 0.0000, 0.0000, 0.0000, 0.8026, 0.0000, 0.0000, 0.0000, 0.2029],
         [0.8416, 0.0000, 0.0000, 0.0000, 1.4524, 0.0000, 0.0000, 0.0000, 1.6004],
    

In [ ]:
model(x_t)

tensor([[-0.0010,  0.0057, -0.0049],
        [ 0.0060, -0.0232,  0.0303],
        [ 0.0243, -0.0298,  0.0184],
        [ 0.0082, -0.0110, -0.0127],
        [ 0.0624,  0.0886,  0.0515],
        [ 0.0131, -0.0119,  0.0454],
        [ 0.0011, -0.0057, -0.0103],
        [-0.0072, -0.0318,  0.0002],
        [ 0.0222, -0.0437,  0.0027],
        [ 0.0261, -0.0898,  0.0167],
        [-0.0071, -0.0392,  0.0054],
        [ 0.0096, -0.0153, -0.0128],
        [ 0.0551,  0.0379, -0.0203],
        [-0.0736, -0.0711,  0.0804],
        [ 0.0517, -0.1048,  0.0208],
        [ 0.0196, -0.0182, -0.0116],
        [ 0.0393,  0.0686,  0.0495],
        [-0.0359,  0.0636, -0.0047],
        [-0.0214, -0.0323,  0.0143],
        [ 0.0034,  0.0368,  0.0268],
        [ 0.0216, -0.0087,  0.0588],
        [-0.0440,  0.0066, -0.0344],
        [ 0.0092,  0.0079, -0.0196],
        [ 0.0155, -0.0014, -0.0429],
        [-0.0465, -0.0121, -0.0150],
        [ 0.0074,  0.0367,  0.0235],
        [-0.0160, -0.0314, -0.0653],
 